In [2]:
### Для проверки данного задания вам потребуется программное обеспечение Anaconda, подходящее под вашу версию операционной системы
### И Jupyter Notebook. который входит в пакет Anaconda


###1) Соединение БД с Python
###Если вдруг кого не было на семинарах и еще не грузил эту штуку, для MAC OS пишется по другому см коммент ниже
pip install psycopg2
###pip install psycopg2-binary
###Ошибка так как уже все устанавлено было

SyntaxError: invalid syntax (<ipython-input-2-3db8d7e0a752>, line 3)

In [3]:
####импортируем всякие нужные штучки
import psycopg2 # драйвер для postgresql
import getpass  # прячем параль, чтобы не писать его в параметрах
### создаем соединение
conn=psycopg2.connect(user='postgres',
                      database='postgres',##название базы данных мб другое, как и другие параметры соединения, измените под свои
                      host='localhost',
                      port='5432',
                      password=getpass.getpass('Insert password: ')
                        )
print(conn.get_dsn_parameters())###Проверка, что все норм подключилось


Insert password: ········
{'user': 'postgres', 'dbname': 'postgres', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


In [4]:
# 2 пункт, чтобы вывести нужные данные из БД, неверная попытка
import pandas as pd
query="SELECT * FROM bond_description_task  WHERE \"Status\"='В обращении' AND \"CouponType\"='Постоянный' OR \"CouponType\"='Дисконт' OR \"RateTypeNameRus_NRD\"='Нулевой';"
data=pd.read_sql_query(query,conn)
data.head(10)

,"ISIN, RegCode, NRDCode",FinToolType,SecurityType,SecurityKind,CouponType,RateTypeNameRus_NRD,CouponTypeName_NRD,HaveOffer,AmortisedMty,MaturityGroup,...,CurrentFaceValue_NRD,BorrowerName,BorrowerOKPO,BorrowerSector,BorrowerUID,IssuerName,IssuerName_NRD,IssuerOKPO,NumGuarantors,EndMtyDate
0,XS1092320099,Облигация,ЕвроКорп,Классические,Постоянный,с фиксированной ставкой,Полугодовой,True,False,Среднесрочные,...,1.000000e+03,4finance AS,NaN,None,95365,4finance S.A,4finance S.A.,NaN,9,2019-08-14
1,XS1076087375,Облигация,ЕвроКорп,Ноты участия в кредите,Постоянный,с фиксированной ставкой,Ежегодный,False,False,Краткосрочные,...,1.000000e+03,ABH Financial,NaN,None,87458,Alfa Holding Issuance,Alfa Holding Issuance plc,NaN,0,2017-06-10
2,XS0974469206,Облигация,ЕвроКорп,Ноты участия в кредите,Постоянный,с фиксированной ставкой,Полугодовой,False,False,Среднесрочные,...,1.000000e+03,Borets International,NaN,None,94896,Borets Finance,Borets Finance Limited,NaN,5,2018-09-26
3,XS0868348987,Облигация,ЕвроКорп,Классические,Постоянный,с фиксированной ставкой,Полугодовой,False,False,Краткосрочные,...,1.000000e+05,Caterpillar FCS,NaN,None,7838,Caterpillar InternFinance,Caterpillar International Finance Limited,NaN,1,2015-12-20
4,RU000A0JRHE6,Облигация,Корп,Классические,Постоянный,с фиксированной ставкой,Ежегодный,False,False,Среднесрочные,...,1.000000e+09,EBRD Bank,NaN,None,6582,EBRD Bank,Европейский Банк Реконструкции и Развития,NaN,0,2016-06-15
5,XS0918604496,Облигация,ЕвроКорп,Классические,Постоянный,с фиксированной ставкой,Полугодовой,False,False,Среднесрочные,...,1.000000e+03,Eurasia Drilling,NaN,None,93311,EDC Finance,EDC Finance Limited,NaN,3,2020-04-17
6,XS0995845566,Облигация,ЕвроКорп,Ноты участия в кредите,Постоянный,с фиксированной ставкой,Полугодовой,False,False,Среднесрочные,...,1.000000e+03,Hacienda Investments,NaN,None,94944,DME Airport,DME Airport Limited,NaN,8,2018-11-26
7,RU000A0JSLP2,Облигация,Корп,Классические,Постоянный,с фиксированной ставкой,Ежегодный,False,False,Среднесрочные,...,1.000000e+07,IFC,NaN,None,6925,IFC,МФК,NaN,0,2017-11-26
8,RU000A0JSLQ0,Облигация,Корп,Классические,Постоянный,с фиксированной ставкой,Ежегодный,False,False,Среднесрочные,...,1.000000e+07,IFC,NaN,None,6925,IFC,МФК,NaN,0,2017-11-26
9,RU000A0JSLR8,Облигация,Корп,Классические,Постоянный,с фиксированной ставкой,Ежегодный,False,False,Среднесрочные,...,1.000000e+07,IFC,NaN,None,6925,IFC,МФК,NaN,0,2017-11-26


In [8]:
###Финальная попытка вывести данные из бд (Необходимые как для отображения в матрице, так и для расчетов)
import pandas as pd
query="SELECT DISTINCT bond_description_task.\"ISINCode\",bond_description_task.\"IssuerName\", bond_description_task.\"HaveOffer\", bond_description_task.\"CouponType\","\
"bond_description_task.\"SecurityType\", quotes_task.\"CPN\","\
"quotes_task.\"CPN_DATE\", quotes_task.\"FACEVALUE\", "\
"quotes_task.\"MAT_DATE\" FROM bond_description_task INNER JOIN quotes_task ON "\
"bond_description_task.\"ISINCode\"=quotes_task.\"ISIN\" WHERE \"Status\"!='Погашен' and \"Status\"!='Размещается' "\
"AND \"HaveDefault\"='0'"
data=pd.read_sql_query(query,conn)
###ВЫводим первые 10 строк из запроса для проверки
data.head(10)

,ISINCode,IssuerName,HaveOffer,CouponType,SecurityType,BUYBACKDATE,CPN,CPN_DATE,CPN_PERIOD,FACEVALUE,MAT_DATE
0,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,38.15,2011-02-24,182.0,1000.0,2016-02-18
1,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,38.15,2011-08-25,182.0,1000.0,2016-02-18
2,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,38.15,2012-02-23,182.0,1000.0,2016-02-18
3,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,38.15,2012-08-23,182.0,1000.0,2016-02-18
4,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,36251.00,2012-08-23,182.0,1000.0,2016-02-18
5,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,2009-08-27,NaN,1000.0,2016-02-18
6,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,2010-02-25,NaN,1000.0,2016-02-18
7,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,2010-08-26,NaN,1000.0,2016-02-18
8,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,2011-02-24,NaN,1000.0,2016-02-18
9,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,Корп,2013-02-21,NaN,None,NaN,1000.0,2016-02-18


In [51]:
###Строим функцию, 1 аргумент имя эмитента, 2 аргумент дата на которую строим поток платежей, 3 соединение с базой данной, с помощью которого собираем инфу
def my_fun(issuername, cpndate, conn):
    ####Импортируем необходимые библиотеки
    import pandas as pd
    import numpy as np
    ####Запрос из пункта 2 дз вставляем здесь
    query="SELECT DISTINCT bond_description_task.\"ISINCode\",bond_description_task.\"IssuerName\","\
    "bond_description_task.\"HaveOffer\", bond_description_task.\"CouponType\", quotes_task.\"CPN\", quotes_task.\"CPN_DATE\", "\
    "quotes_task.\"FACEVALUE\", quotes_task.\"MAT_DATE\" FROM bond_description_task "\
    "INNER JOIN quotes_task ON bond_description_task.\"ISINCode\"=quotes_task.\"ISIN\" "\
    "WHERE \"Status\"!='Погашен' and \"Status\"!='Размещается' and \"HaveDefault\"='0'"\
    "AND ""bond_description_task.\"IssuerName\"="+issuername+" AND quotes_task.\"CPN_DATE\">="+cpndate+""###больше либо равно, чтобы с заданной даты платежи были
    ####Заносим данные из запроса в переменнную data
    data=pd.read_sql_query(query,conn)
    
    ###Отбираем облигации с фиксированным купоном
    coupon=data[['ISINCode','IssuerName','CPN','CPN_DATE','MAT_DATE','FACEVALUE']][(data['CouponType']=='Постоянный')|(data['CouponType']=='Фиксированный')]
    coupon['CPN'].fillna(0, inplace=True)
    import numpy as np
    ###Рассчитываем поток выплат для облигаций с фиксированным купоном
    coupon['CASHFLOW']=np.where(coupon['CPN_DATE']==coupon['MAT_DATE'], coupon['CPN']+coupon['FACEVALUE'], coupon['CPN'])
    coupon_table=coupon[['IssuerName','ISINCode','CPN_DATE','CASHFLOW']]
    ####Строим сводную таблицу для графика платежей
    coupon_table=coupon_table.pivot_table(index='ISINCode', columns='CPN_DATE', values='CASHFLOW', aggfunc='sum').reset_index().rename_axis(None, axis=1)
    ###Конец нужен для красивого отображения таблицы
    ####В те даты, когда у облигации нет выплат заполняем нулем
    coupon_table=coupon_table.fillna(0)
    ###выводим результаты функции
    results={'IssuerName': issuername, 'CPN_DATE': cpndate, 'COUPON_TABLE': coupon_table}
    return results

In [53]:

login={'user':'postgres', 'database':'postgres', 'host':'localhost', 'port':'5432'}
####Задаем параметры для функции и наслаждаемся результатом
issuername ='\'Магнит\''
cpndate ='\'2009-12-28\''
####ну и выводим результат
show=my_fun(issuername, cpndate, conn)
show['COUPON_TABLE']

,ISINCode,2013-08-27,2013-10-01,2014-02-25,2014-04-01,2014-08-26,2014-09-30,2015-02-24,2015-03-31,2015-08-25,2015-09-29,2015-11-13,2016-02-23,2016-03-29
0,RU000A0JTP09,42.38,0.00,42.38,0.00,42.38,0.00,42.38,0.00,42.38,0.00,0.00,1042.38,0.00
1,RU000A0JTT21,0.00,41.88,0.00,41.88,0.00,41.88,0.00,41.88,0.00,41.88,0.00,0.00,1041.88
2,RU000A0JTT39,0.00,41.88,0.00,41.88,0.00,41.88,0.00,41.88,0.00,41.88,0.00,0.00,1041.88
3,RU000A0JVE81,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,60.33,0.00,0.00
4,RU000A0JVE99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,60.33,0.00,0.00


In [54]:
###Бонусное задание
def my_fun2(issuername, cpndate, db_login):
    import pandas as pd
    import numpy as np
    conn=psycopg2.connect(user=db_login['user'],
                      database=db_login['database'],
                      host=db_login['host'],
                      port=db_login['port'],
                      password=getpass.getpass('Insert password: '))
    query="SELECT DISTINCT bond_description_task.\"ISINCode\",bond_description_task.\"IssuerName\","\
    "bond_description_task.\"HaveOffer\", bond_description_task.\"CouponType\", quotes_task.\"CPN\", quotes_task.\"CPN_DATE\", "\
    "quotes_task.\"FACEVALUE\", quotes_task.\"MAT_DATE\" FROM bond_description_task "\
    "INNER JOIN quotes_task ON bond_description_task.\"ISINCode\"=quotes_task.\"ISIN\" "\
    "WHERE \"Status\"!='Погашен' and \"Status\"!='Размещается' and \"HaveDefault\"='0'"\
    "AND ""bond_description_task.\"IssuerName\"="+issuername+" AND quotes_task.\"CPN_DATE\">="+cpndate+""
    data=pd.read_sql_query(query,conn)
    
    ###Отбираем облигации с фиксированным купоном
    coupon=data[['ISINCode','IssuerName','CPN','CPN_DATE','MAT_DATE','FACEVALUE']][(data['CouponType']=='Постоянный')|(data['CouponType']=='Фиксированный')]
    coupon['CPN'].fillna(0, inplace=True)
    
    ###Рассчитываем поток выплат для облигаций с фиксированным купоном
    coupon['CASHFLOW']=np.where(coupon['CPN_DATE']==coupon['MAT_DATE'], coupon['CPN']+coupon['FACEVALUE'], coupon['CPN'])
    coupon_table=coupon[['IssuerName','ISINCode','CPN_DATE','CASHFLOW']]
    ####Строим сводную таблицу для графика платежей
    coupon_table=coupon_table.pivot_table(index='ISINCode', columns='CPN_DATE', values='CASHFLOW', aggfunc='sum').reset_index().rename_axis(None, axis=1)
    ###Конец нужен для красивого отображения таблицы
    ####В те даты, когда у облигации нет выплат заполняем нулем
    coupon_table=coupon_table.fillna(0)
    ###выводим результаты функции
    results={'IssuerName': issuername, 'CPN_DATE': cpndate, 'COUPON_TABLE': coupon_table}
    return results
